In [1]:
from sklearn.model_selection import train_test_split
# =============================================================================
# from k_means import build_color_array
# 
# =============================================================================
from sklearn.utils import class_weight
from sklearn import svm
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import SVC
import pandas as pd
import pickle
import warnings
warnings.filterwarnings("ignore")
import os

In [2]:
#import dataframe and visualize

path = 'normalized.pickle'
df = pd.read_pickle(os.path.join(r'C:\Users\Teresa\Desktop\TESE\Textural Analysis', path))

df

,Area,Image,Mean Green,Mean Intensity,Mean Red,Normalized Mean Green,Normalized Mean Red,Normalized Total Green,Normalized Total Red,Nucleus Patch,...,Equivalent Diameter,Major Axis Length,Minor Axis Length,Max Intensity,Min Intensity,Orientation,Perimeter,Solidity,Entropy,Circularity
0,4155,All_FS2_1_40xoil_60Zs_deconv_10_Mask.tif.png.png,12.619976,125.657280,46.411793,0.262386,0.964963,0.262386,0.964963,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",...,72.734519,93.715394,56.642633,255,44,0.884948,248.350288,0.983199,5.579251,0.846548
3,4760,All_FS2_1_40xoil_60Zs_deconv_10_Mask.tif.png.png,29.452311,120.828992,37.138025,0.621369,0.783518,0.621369,0.783518,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",...,77.849985,93.066080,65.323570,255,40,1.412099,258.894444,0.992287,6.150013,0.892424
4,5014,All_FS2_1_40xoil_60Zs_deconv_10_Mask.tif.png.png,6.779418,114.041284,86.455325,0.078175,0.996940,0.078175,0.996940,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",...,79.900082,88.914160,72.074866,255,46,-1.500195,261.722871,0.990518,5.924569,0.919837
6,5503,All_FS2_1_40xoil_60Zs_deconv_10_Mask.tif.png.png,10.632564,106.558968,73.093949,0.143949,0.989585,0.143949,0.989585,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",...,83.705658,89.324967,78.566344,255,39,-0.744969,271.764502,0.993142,5.826534,0.936319
7,5800,All_FS2_1_40xoil_60Zs_deconv_10_Mask.tif.png.png,7.822759,116.757931,120.578621,0.064741,0.997902,0.064741,0.997902,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",...,85.934797,88.987739,83.083881,255,48,-1.071380,279.764502,0.992131,5.580421,0.931221
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8607,5145,All_Id_FS6_3_40xoil_60Zs_deconv_9_Mask.tif.png...,77.881633,152.714480,4.357629,0.998438,0.055865,0.998438,0.055865,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",...,80.937120,91.817259,71.901276,255,46,-1.451276,273.764502,0.985632,5.352308,0.862663
8614,4802,All_Id_FS6_3_40xoil_60Zs_deconv_9_Mask.tif.png...,52.264681,136.686381,2.954602,0.998406,0.056441,0.998406,0.056441,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",...,78.192687,84.966272,71.992936,255,57,0.596904,258.107648,0.986239,5.435626,0.905796
8615,3566,All_Id_FS6_3_40xoil_60Zs_deconv_9_Mask.tif.png...,17.987942,127.823331,39.335109,0.415878,0.909420,0.415878,0.909420,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",...,67.382284,77.833736,58.528837,255,52,-1.448471,228.409163,0.978864,5.656561,0.858943
8617,4908,All_Id_FS6_3_40xoil_60Zs_deconv_9_Mask.tif.png...,51.481866,152.560921,7.560106,0.989389,0.145292,0.989389,0.145292,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",...,79.050994,79.792635,78.405947,255,59,-1.557158,261.622366,0.985542,6.055117,0.901083


In [3]:
#input dapi features
matrix = df.as_matrix(columns = ['norm_area', 'norm_intensity'])
X = matrix.astype(float).reshape(matrix.shape)

#output FUCCI labels
y_aux = df.as_matrix(columns = ['Automatic Label'])
y = y_aux.astype(float).reshape(y_aux.shape)
y = y.ravel()



X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)


y_train = y_train.ravel()
y_test = y_test.ravel()

In [4]:
# =============================================================================
# # Set the parameters by cross-validation
 tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-2, 1e-3, 1e-4, 1e-5],
                      'C': [0.001, 0.10, 0.1, 10, 25, 50, 100, 1000]},
                     {'kernel': ['sigmoid'], 'gamma': [1e-2, 1e-3, 1e-4, 1e-5],
                      'C': [0.001, 0.10, 0.1, 10, 25, 50, 100, 1000]},
                     {'kernel': ['linear'], 'C': [0.001, 0.10, 0.1, 10, 25, 50, 100, 1000]},
                     {'kernel': ['poly'], 'C': [0.001, 0.10, 0.1, 10, 25, 50, 100, 1000],
                      'degree': [1, 2, 3, 4, 5]}
                    ]
# =============================================================================



In [ ]:
scores = ['f1_macro']

#scores = {'precision': 'precision_macro', 'recall': 'recall_macro', 'f1score': 'f1_macro'}

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()
    
    clf = GridSearchCV(SVC(class_weight = 'balanced'), tuned_parameters, cv=5,
                       scoring= score)
    clf.fit(X_train, y_train)
    
    print("Best parameters set found on development set:")
    print()
    print(clf.best_params)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()
    
    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test, clf.predict(X_test)
    print(classification_report(y_true, y_pred))
    print()
    
    


# Tuning hyper-parameters for f1_macro

